# 7장 데이터 전처리

> 데이터를 전처리하는 내용을 다룹니다.
- author: "Chansung Park"
- toc: true
- comments: true
- permalink: /chapter7/
- badges: true
- sticky_rank: 7

In [1]:
#hide

!pip install fastai==1.0.61
!pip install yellowbrick==0.9
!pip install scikit-learn==0.21.0
!pip install pyjanitor==0.16.6
!pip install category_encoders==2.0.0
!pip install pandas==0.23.4

     |████████████████████████████████| 286kB 5.8MB/s 
     |████████████████████████████████| 12.8MB 308kB/s 
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: yellowbrick 0.9.1
    Uninstalling yellowbrick-0.9.1:
      Successfully uninstalled yellowbrick-0.9.1


     |████████████████████████████████| 6.7MB 4.1MB/s 
ERROR: umap-learn 0.5.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.0 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 2.3MB 6.6MB/s 
  Created wheel for sphinxcontrib-fulltoc: filename=sphinxcontrib_fulltoc-1.2.0-py2.py3-none-any.whl size=8715 sha256=d970e5aeefc98765ff3277c1604dea9e692b3e427bcd8025690c42da4ea6e8b9
  Stored in directory: /root/.cache/pip/wheels/74/30/b2/f2da9acb18f37b571f3c17aa03ee6f1ac555514d99d76b9af0
Successfully built sphinxcontrib-fulltoc


     |████████████████████████████████| 92kB 3.3MB/s 
     |████████████████████████████████| 8.8MB 5.7MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.23.4 which is incompatible.
ERROR: pyjanitor 0.16.6 has requirement pandas>=0.24.0, but you'll have pandas 0.23.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.4 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.5 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.23.4 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 0.23.4 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.23.4 which is incompatible.


In [1]:
#hide

import warnings
warnings.filterwarnings('ignore')

In [2]:
#hide

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
  ensemble,
  preprocessing,
  tree
)
from sklearn.metrics import (
  auc,
  confusion_matrix,
  roc_auc_score,
  roc_curve
)
from sklearn.model_selection import (
  train_test_split,
  StratifiedKFold
)
from yellowbrick.classifier import (
  ConfusionMatrix,
  ROCAUC
)
from yellowbrick.model_selection import (
  LearningCurve
)

In [3]:
#hide

import sklearn
import re
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute._base import SimpleImputer as Imputer
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.ensemble import forest
from sklearn.tree import export_graphviz

def set_plot_sizes(sml, med, big):
    plt.rc('font', size=sml)          # controls default text sizes
    plt.rc('axes', titlesize=sml)     # fontsize of the axes title
    plt.rc('axes', labelsize=med)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=sml)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=sml)    # fontsize of the tick labels
    plt.rc('legend', fontsize=sml)    # legend fontsize
    plt.rc('figure', titlesize=big)  # fontsize of the figure title

def parallel_trees(m, fn, n_jobs=8):
        return list(ProcessPoolExecutor(n_jobs).map(fn, m.estimators_))

def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    """ Draws a representation of a random forest in IPython.
    Parameters:
    -----------
    t: The tree you wish to draw
    df: The data used to train the tree. This is used to get the names of the features.
    """
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

def combine_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
              seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)

def get_sample(df,n):
    """ Gets a random sample of n rows from df, without replacement.
    Parameters:
    -----------
    df: A pandas data frame, that you wish to sample from.
    n: The number of rows you wish to sample.
    Returns:
    --------
    return value: A random sample of n rows of df.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    >>> get_sample(df, 2)
       col1 col2
    1     2    b
    2     3    a
    """
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

def add_datepart(df, fldnames, drop=True, time=False, errors="raise"):	
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string or list of strings that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    >>>df2 = pd.DataFrame({'start_date' : pd.to_datetime(['3/11/2000','3/13/2000','3/15/2000']),
                            'end_date':pd.to_datetime(['3/17/2000','3/18/2000','4/1/2000'],infer_datetime_format=True)})
    >>>df2
        start_date	end_date    
    0	2000-03-11	2000-03-17
    1	2000-03-13	2000-03-18
    2	2000-03-15	2000-04-01
    >>>add_datepart(df2,['start_date','end_date'])
    >>>df2
    	start_Year	start_Month	start_Week	start_Day	start_Dayofweek	start_Dayofyear	start_Is_month_end	start_Is_month_start	start_Is_quarter_end	start_Is_quarter_start	start_Is_year_end	start_Is_year_start	start_Elapsed	end_Year	end_Month	end_Week	end_Day	end_Dayofweek	end_Dayofyear	end_Is_month_end	end_Is_month_start	end_Is_quarter_end	end_Is_quarter_start	end_Is_year_end	end_Is_year_start	end_Elapsed
    0	2000	    3	        10	        11	        5	            71	            False	            False	                False	                False	                False	            False	            952732800	    2000	    3	        11	        17	    4	            77	            False	            False	            False	            False	                False	        False	            953251200
    1	2000	    3	        11	        13	        0	            73	            False	            False	                False	                False               	False           	False           	952905600     	2000       	3	        11      	18  	5           	78          	False	            False           	False           	False               	False          	False           	953337600
    2	2000	    3	        11	        15	        2           	75          	False           	False               	False               	False               	False               False           	953078400      	2000    	4          	13      	1   	5           	92          	False           	True            	False           	True                	False          	False           	954547200
    """
    if isinstance(fldnames,str): 
        fldnames = [fldnames]
    for fldname in fldnames:
        fld = df[fldname]
        fld_dtype = fld.dtype
        if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            fld_dtype = np.datetime64

        if not np.issubdtype(fld_dtype, np.datetime64):
            df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
        targ_pre = re.sub('[Dd]ate$', '', fldname)
        attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
                'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
        if time: attr = attr + ['Hour', 'Minute', 'Second']
        for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
        df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
        if drop: df.drop(fldname, axis=1, inplace=True)

def is_date(x): return np.issubdtype(x.dtype, np.datetime64)

def train_cats(df):
    """Change any columns of strings in a panda's dataframe to a column of
    categorical values. This applies the changes inplace.
    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category
    """
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.
    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.
    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category {a : 1, b : 2}
    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)
           col1 col2
        0     1    b
        1     2    a
        2     3    a
    now the type of col is category {a : 1, b : 2}
    """
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = c.astype('category').cat.as_ordered()
            df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)

def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.
    Parameters:
    -----------
    df: The data frame that will be changed.
    col: The column of data to fix by filling in missing data.
    name: The name of the new filled column in df.
    na_dict: A dictionary of values to create na's of and the value to insert. If
        name is not a key of na_dict the median will fill any missing data. Also
        if name is not a key of na_dict and there is no missing data in col, then
        no {name}_na column is not created.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2
    >>> fix_missing(df, df['col1'], 'col1', {})
    >>> df
       col1 col2 col1_na
    0     1    5   False
    1     2    2    True
    2     3    2   False
    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2
    >>> fix_missing(df, df['col2'], 'col2', {})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2
    >>> df = pd.DataFrame({'col1' : [1, np.NaN, 3], 'col2' : [5, 2, 2]})
    >>> df
       col1 col2
    0     1    5
    1   nan    2
    2     3    2
    >>> fix_missing(df, df['col1'], 'col1', {'col1' : 500})
    >>> df
       col1 col2 col1_na
    0     1    5   False
    1   500    2    True
    2     3    2   False
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def numericalize(df, col, name, max_n_cat):
    """ Changes the column col from a categorical type to it's integer codes.
    Parameters:
    -----------
    df: A pandas dataframe. df[name] will be filled with the integer codes from
        col.
    col: The column you wish to change into the categories.
    name: The column name you wish to insert into df. This column will hold the
        integer codes.
    max_n_cat: If col has more categories than max_n_cat it will not change the
        it to its integer codes. If max_n_cat is None, then col will always be
        converted.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category { a : 1, b : 2}
    >>> numericalize(df, df['col2'], 'col3', None)
       col1 col2 col3
    0     1    a    1
    1     2    b    2
    2     3    a    1
    """
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

def scale_vars(df, mapper):
    warnings.filterwarnings('ignore', category=sklearn.exceptions.DataConversionWarning)
    if mapper is None:
        map_f = [([n],StandardScaler()) for n in df.columns if is_numeric_dtype(df[n])]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return mapper

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe. For each column of df 
    which is not in skip_flds nor in ignore_flds, na values are replaced by the
    median value of the column.
    Parameters:
    -----------
    df: The data frame you wish to process.
    y_fld: The name of the response variable
    skip_flds: A list of fields that dropped from df.
    ignore_flds: A list of fields that are ignored during processing.
    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)
    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.
    preproc_fn: A function that gets applied to df.
    max_n_cat: The maximum number of categories to break into dummy values, instead
        of integer codes.
    subset: Takes a random subset of size subset from df.
    mapper: If do_scale is set as True, the mapper variable
        calculates the values used for scaling of variables during training time (mean and standard deviation).
    Returns:
    --------
    [x, y, nas, mapper(optional)]:
        x: x is the transformed version of df. x will not have the response variable
            and is entirely numeric.
        y: y is the response variable
        nas: returns a dictionary of which nas it created, and the associated median.
        mapper: A DataFrameMapper which stores the mean and standard deviation of the corresponding continuous
        variables which is then used for scaling of during test-time.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category { a : 1, b : 2}
    >>> x, y, nas = proc_df(df, 'col1')
    >>> x
       col2
    0     1
    1     2
    2     1
    >>> data = DataFrame(pet=["cat", "dog", "dog", "fish", "cat", "dog", "cat", "fish"],
                 children=[4., 6, 3, 3, 2, 3, 5, 4],
                 salary=[90, 24, 44, 27, 32, 59, 36, 27])
    >>> mapper = DataFrameMapper([(:pet, LabelBinarizer()),
                          ([:children], StandardScaler())])
    >>>round(fit_transform!(mapper, copy(data)), 2)
    8x4 Array{Float64,2}:
    1.0  0.0  0.0   0.21
    0.0  1.0  0.0   1.88
    0.0  1.0  0.0  -0.63
    0.0  0.0  1.0  -0.63
    1.0  0.0  0.0  -1.46
    0.0  1.0  0.0  -0.63
    1.0  0.0  0.0   1.04
    0.0  0.0  1.0   0.21
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = pd.Categorical(df[y_fld]).codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

def get_nn_mappers(df, cat_vars, contin_vars):
    # Replace nulls with 0 for continuous, "" for categorical.
    for v in contin_vars: df[v] = df[v].fillna(df[v].max()+100,)
    for v in cat_vars: df[v].fillna('#NA#', inplace=True)

    # list of tuples, containing variable and instance of a transformer for that variable
    # for categoricals, use LabelEncoder to map to integers. For continuous, standardize
    cat_maps = [(o, LabelEncoder()) for o in cat_vars]
    contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [60]:
#hide

url = "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"
df = pd.read_excel(url)
orig_df = df

mask = df.isnull().any(axis=1)

df = df.drop(
    columns=["ticket",
             "home.dest",
             "boat",
             "body"]
)

# df = pd.get_dummies(df)
# df = df.drop(columns="sex_male")
# df = pd.get_dummies(df, drop_first=True)

# y = df.survived
# X = df.drop(columns="survived")

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.3, random_state=42)

# from sklearn.experimental import (
#   enable_iterative_imputer,
# )
# from sklearn import impute

# num_cols = [
#   "pclass",
#   "age",
#   "sibsp",
#   "parch",
#   "fare",
#   "sex_female",
# ]

# imputer = impute.IterativeImputer()
# imputed = imputer.fit_transform(
#   X_train[num_cols]
# )
# X_train.loc[:, num_cols] = imputed
# imputed = imputer.transform(X_test[num_cols])
# X_test.loc[:, num_cols] = imputed

# meds = X_train.median()
# X_train = X_train.fillna(meds)
# X_test = X_test.fillna(meds)

# cols = "pclass,age,sibsp,fare".split(",")
# sca = preprocessing.StandardScaler()

# X_train = sca.fit_transform(X_train)
# X_train = pd.DataFrame(X_train[:, :4], columns=cols)
# X_test = sca.transform(X_test)
# X_test = pd.DataFrame(X_test[:, :4], columns=cols)

# def tweak_titanic(df):
#   df = df.drop(
#     columns=[
#       "name",
#       "ticket",
#       "home.dest",
#       "boat",
#       "body",
#       "cabin",
#     ]
#   ).pipe(pd.get_dummies, drop_first=True)
#   return df

# def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
#   y = df[y_col]
#   X = df.drop(columns=y_col)
#   X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=size, random_state=42
#   )
#   cols = X.columns
#   num_cols = [
#     "pclass",
#     "age",
#     "sibsp",
#     "parch",
#     "fare",
#   ]

#   fi = impute.IterativeImputer()

#   X_train.loc[:, num_cols] = fi.fit_transform(X_train[num_cols])
#   X_test.loc[:, num_cols] = fi.transform(X_test[num_cols])

#   if std_cols:
#     std = preprocessing.StandardScaler()
#     X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
#     X_test.loc[:, std_cols] = std.transform(X_test[std_cols])

#   return X_train, X_test, y_train, y_test  

# ti_df = tweak_titanic(orig_df)
# std_cols = "pclass,age,sibsp,fare".split(",")

# X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)  

# from sklearn.dummy import DummyClassifier
# bm = DummyClassifier()
# bm.fit(X_train, y_train)
# bm.score(X_test, y_test) # 정확도

# from sklearn import metrics
# metrics.precision_score(y_test, bm.predict(X_test))

# X = pd.concat([X_train, X_test])
# y = pd.concat([y_train, y_test])

In [61]:
X2 = pd.DataFrame(
  {
    "a": range(5),
    "b": [-100, -50, 0, 200, 1000],
  }
)
X2

,a,b
0,0,-100
1,1,-50
2,2,0
3,3,200
4,4,1000


# 7.1 표준화

In [62]:
from sklearn import preprocessing
std = preprocessing.StandardScaler()
std.fit_transform(X2)

array([[-1.41421356, -0.75995002],
       [-0.70710678, -0.63737744],
       [ 0.        , -0.51480485],
       [ 0.70710678, -0.02451452],
       [ 1.41421356,  1.93664683]])

In [63]:
std.scale_

array([  1.41421356, 407.92156109])

In [64]:
std.mean_

array([  2., 210.])

In [65]:
std.var_

array([2.000e+00, 1.664e+05])

In [66]:
X_std = (X2 - X2.mean()) / X2.std()
X_std

,a,b
0,-1.264911,-0.679720
1,-0.632456,-0.570088
2,0.000000,-0.460455
3,0.632456,-0.021926
4,1.264911,1.732190


In [67]:
X_std.mean()

a    4.440892e-17
b    0.000000e+00
dtype: float64

In [68]:
X_std.std()

a    1.0
b    1.0
dtype: float64

In [69]:
X3 = X2.copy()
scale_vars(X3, mapper=None) #fastai function locally imported
X3.std()

a    1.118034
b    1.118034
dtype: float64

In [70]:
X3.mean()

a    0.000000e+00
b    4.440892e-17
dtype: float64

# 7.2 범위 조정

In [71]:
from sklearn import preprocessing
mms = preprocessing.MinMaxScaler()
mms.fit(X2)
mms.transform(X2)

array([[0.        , 0.        ],
       [0.25      , 0.04545455],
       [0.5       , 0.09090909],
       [0.75      , 0.27272727],
       [1.        , 1.        ]])

In [72]:
(X2 - X2.min()) / (X2.max() - X2.min())

,a,b
0,0.00,0.000000
1,0.25,0.045455
2,0.50,0.090909
3,0.75,0.272727
4,1.00,1.000000


# 7.3 더미 변수

In [73]:
X_cat = pd.DataFrame(
  {
    "name": ["George", "Paul"],
    "inst": ["Bass", "Guitar"],
  }
)

X_cat

,name,inst
0,George,Bass
1,Paul,Guitar


In [74]:
pd.get_dummies(X_cat, drop_first=True)

,name_Paul,inst_Guitar
0,0,0
1,1,1


In [75]:
import janitor as jn

X_cat2 = pd.DataFrame(
  {
    "A": [1, None, 3],
    "names": [
        "Fred,George",
        "George",
        "John,Paul",
      ],
  }
)

jn.expand_column(X_cat2, "names", sep=",")

,A,names,Fred,George,John,Paul
0,1.0,"Fred,George",1,1,0,0
1,NaN,George,0,1,0,0
2,3.0,"John,Paul",0,0,1,1


# 7.4 레이블 인코더

In [76]:
from sklearn import preprocessing
lab = preprocessing.LabelEncoder()
lab.fit_transform(X_cat['inst'])

array([0, 1])

In [77]:
lab.inverse_transform([1, 1, 0])

array(['Guitar', 'Guitar', 'Bass'], dtype=object)

In [78]:
X_cat.name.astype(
  "category"
).cat.as_ordered().cat.codes + 1

0    1
1    2
dtype: int8

# 7.5 프리퀀시 인코딩

In [79]:
mapping = X_cat.name.value_counts()
X_cat.name.map(mapping)

0    1
1    1
Name: name, dtype: int64

# 7.6 문자열에서 범주 가져오기

In [80]:
df

,pclass,survived,name,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,211.3375,B5,S
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,151.5500,C22 C26,S
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,151.5500,C22 C26,S
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,151.5500,C22 C26,S
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,151.5500,C22 C26,S
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,26.5500,E12,S
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,77.9583,D7,S
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,0.0000,A36,S
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,51.4792,C101,S
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,49.5042,NaN,C


In [81]:
from collections import Counter
c = Counter()

def triples(val):
  for i in range(len(val)):
    c[val[i : i + 3]] += 1

df.name.apply(triples)
c.most_common(10)

[(', M', 1282),
 (' Mr', 954),
 ('r. ', 830),
 ('Mr.', 757),
 ('s. ', 460),
 ('n, ', 320),
 (' Mi', 283),
 ('iss', 261),
 ('ss.', 261),
 ('Mis', 260)]

In [82]:
df.name.str.extract(
  "([A-Za-z]+)\.", expand=False
).head()

0      Miss
1    Master
2      Miss
3        Mr
4       Mrs
Name: name, dtype: object

In [83]:
df.name.str.extract(
  "([A-Za-z]+)\.", expand=False
).value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Ms            2
Mlle          2
Major         2
Don           1
Jonkheer      1
Sir           1
Capt          1
Countess      1
Mme           1
Dona          1
Lady          1
Name: name, dtype: int64

# 7.7 그 밖의 범주형 인코딩

In [84]:
import category_encoders as ce
he = ce.HashingEncoder(verbose=1)
he.fit_transform(X_cat)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,0,0,0,1,0,1,0,0
1,0,2,0,0,0,0,0,0


In [85]:
size_df = pd.DataFrame(
  {
    "name": ["Fred", "John", "Matt"],
    "size": ["small", "med", "xxl"],
  }
)

ore = ce.OrdinalEncoder(
  mapping=[
    {
      "col": "size",
      "mapping": {
        "small": 1,
        "med": 2,
        "lg": 3
      }
    }
  ]
)

ore.fit_transform(size_df)

,name,size
0,Fred,1.0
1,John,2.0
2,Matt,-1.0


In [86]:
def get_title(df):
  return df.name.str.extract(
    "([A-Za-z]+)\.", expand=False
  )

te = ce.TargetEncoder(cols="Title")
te.fit_transform(

df.assign(Title=get_title), df.survived)["Title"].head()

0    0.676923
1    0.508197
2    0.676923
3    0.162483
4    0.786802
Name: Title, dtype: float64

# 7.8 날짜형 데이터의 특징 공학

In [87]:
import numpy as np

dates = pd.DataFrame(
  {
    "A": pd.to_datetime(
      ["9/17/2001", "Jan 1, 2002"]
    )
  }
)

add_datepart(dates, "A")
dates.T

,0,1
AYear,2001,2002
AMonth,9,1
AWeek,38,1
ADay,17,1
ADayofweek,0,1
ADayofyear,260,1
AIs_month_end,False,False
AIs_month_start,False,True
AIs_quarter_end,False,False
AIs_quarter_start,False,True


# 7.9 col_na 특징의 추가

In [88]:
from pandas.api.types import is_numeric_dtype
def fix_missing(df, col, name, na_dict):
  if is_numeric_dtype(col):
    if pd.isnull(col).sum() or (name in na_dict):
      df[name + "_na"] = pd.isnull(col)

      filler = (
          na_dict[name]
          if name in na_dict
          else col.median()
      )

      df[name] = col.fillna(filler)
      na_dict[name] = filler
      return na_dict

data = pd.DataFrame({"A": [0, None, 5, 100]})
fix_missing(data, data.A, "A", {})

{'A': 5.0}

In [89]:
from pandas.api.types import is_numeric_dtype

def fix_missing(df, col, name, na_dict):
  if is_numeric_dtype(col):
    if pd.isnull(col).sum() or (name in na_dict):
      df[name + "_na"] = pd.isnull(col)
      
      filler = (
        na_dict[name]
        if name in na_dict
        else col.median()
      )

      df[name] = col.fillna(filler)
      na_dict[name] = filler
      return na_dict
      
data = pd.DataFrame({"A": [0, None, 5, 100]})
fix_missing(data, data.A, "A", {})

{'A': 5.0}

In [90]:
data

,A,A_na
0,0.0,False
1,5.0,True
2,5.0,False
3,100.0,False


In [91]:
data = pd.DataFrame({"A": [0, None, 5, 100]})
data["A_na"] = data.A.isnull()
data["A"] = data.A.fillna(data.A.median())

# 7.10 수동적 특징 공학

In [92]:
agg = (
  df.groupby("cabin")
  .agg("min,max,mean,sum".split(","))
  .reset_index()
)

agg.columns = [
  "_".join(c).strip("_")
  for c in agg.columns.values
]

agg_df = df.merge(agg, on="cabin")
agg_df

,pclass,survived,name,sex,age,sibsp,parch,fare,cabin,embarked,pclass_min,pclass_max,pclass_mean,pclass_sum,survived_min,survived_max,survived_mean,survived_sum,age_min,age_max,age_mean,age_sum,sibsp_min,sibsp_max,sibsp_mean,sibsp_sum,parch_min,parch_max,parch_mean,parch_sum,fare_min,fare_max,fare_mean,fare_sum
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,211.3375,B5,S,1,1,1,2,1,1,1.000000,2,15.0000,29.0,22.000000,44.0000,0,0,0.000000,0,0,1,0.5,1,211.3375,211.3375,211.337500,422.6750
1,1,1,"Madill, Miss. Georgette Alexandra",female,15.0000,0,1,211.3375,B5,S,1,1,1,2,1,1,1.000000,2,15.0000,29.0,22.000000,44.0000,0,0,0.000000,0,0,1,0.5,1,211.3375,211.3375,211.337500,422.6750
2,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,151.5500,C22 C26,S,1,1,1,4,0,1,0.250000,1,0.9167,30.0,14.479175,57.9167,1,1,1.000000,4,2,2,2.0,8,151.5500,151.5500,151.550000,606.2000
3,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,151.5500,C22 C26,S,1,1,1,4,0,1,0.250000,1,0.9167,30.0,14.479175,57.9167,1,1,1.000000,4,2,2,2.0,8,151.5500,151.5500,151.550000,606.2000
4,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,151.5500,C22 C26,S,1,1,1,4,0,1,0.250000,1,0.9167,30.0,14.479175,57.9167,1,1,1.000000,4,2,2,2.0,8,151.5500,151.5500,151.550000,606.2000
5,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,151.5500,C22 C26,S,1,1,1,4,0,1,0.250000,1,0.9167,30.0,14.479175,57.9167,1,1,1.000000,4,2,2,2.0,8,151.5500,151.5500,151.550000,606.2000
6,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,26.5500,E12,S,1,1,1,1,1,1,1.000000,1,48.0000,48.0,48.000000,48.0000,0,0,0.000000,0,0,0,0.0,0,26.5500,26.5500,26.550000,26.5500
7,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,77.9583,D7,S,1,1,1,1,1,1,1.000000,1,63.0000,63.0,63.000000,63.0000,1,1,1.000000,1,0,0,0.0,0,77.9583,77.9583,77.958300,77.9583
8,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,0.0000,A36,S,1,1,1,1,0,0,0.000000,0,39.0000,39.0,39.000000,39.0000,0,0,0.000000,0,0,0,0.0,0,0.0000,0.0000,0.000000,0.0000
9,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,51.4792,C101,S,1,1,1,3,1,1,1.000000,3,53.0000,59.0,55.666667,167.0000,2,2,2.000000,6,0,0,0.0,0,25.7000,51.4792,42.886133,128.6584
